In [1013]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans


In [1014]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [1015]:
fpl_players_df = pd.read_csv("fpl_historical_data.csv")

In [1016]:
fpl_players_df.head()

,gameweek,id,web_name,team,position,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,region,team_join_date,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,direct_freekicks_order,penalties_order,element_type
0,1,1,Fábio Vieira,Arsenal,Midfielder,True,False,0.0,0.0,438098,0,0,-1,1,0,0.0,0.0,0,Fábio,0.0,False,Has joined Portuguese side FC Porto on loan fo...,2024-08-29T11:06:25.241953Z,5.4,438098.jpg,0.0,False,Ferreira Vieira,0.0,False,NaN,u,3,0,439,0,2775,9,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,683,309,691,314,641,280,0.00,0.00,NaN,NaN,3
1,1,2,G.Jesus,Arsenal,Forward,True,True,100.0,100.0,205651,1,-1,-2,2,1,4.3,4.3,16,Gabriel,3.8,False,NaN,2024-08-25T11:00:05.312580Z,6.8,205651.jpg,1.9,False,Fernando de Jesus,6.3,False,NaN,a,3,26,643122,569985,239371,6350,0.6,3.8,30.0,2022-07-04,349,2,1,1,3,0,0,0,4,0,0,3,104,101.2,67.8,157.0,32.3,3,1.84,0.15,1.99,3.99,0.47,0.04,0.51,1.03,0.77,73,12,281,33,63,13,0.77,0.26,NaN,NaN,4
2,1,3,Gabriel,Arsenal,Defender,True,True,100.0,100.0,226597,1,-1,3,-3,2,4.9,4.9,4,Gabriel,4.4,True,NaN,2024-11-27T10:30:06.325759Z,6.3,226597.jpg,4.9,False,dos Santos Magalhães,27.0,False,NaN,a,3,73,3457416,269285,2088135,19424,0.7,11.6,30.0,2020-09-01,1268,3,2,5,14,0,0,0,2,0,0,7,257,362.2,95.6,240.0,69.8,15,2.15,0.55,2.70,14.18,0.15,0.04,0.19,1.01,0.99,51,16,20,2,10,3,1.06,0.35,NaN,NaN,2
3,1,4,Havertz,Arsenal,Forward,True,True,100.0,100.0,219847,0,0,-1,1,1,4.7,4.7,6,Kai,4.2,False,NaN,2024-11-07T09:00:07.112803Z,7.9,219847.jpg,4.2,False,Havertz,10.1,False,NaN,a,3,67,2298657,78351,2764953,46116,0.5,8.5,80.0,2023-06-28,1390,6,2,5,16,0,0,0,2,0,0,8,250,330.2,199.2,497.0,102.7,16,6.69,1.26,7.95,15.47,0.43,0.08,0.51,1.00,1.04,57,8,43,13,38,9,1.04,0.32,NaN,3.0,4
4,1,5,Hein,Arsenal,Goalkeeper,True,False,0.0,0.0,463748,0,0,0,0,0,0.0,0.0,0,Karl,0.0,False,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,4.0,463748.jpg,0.0,False,Hein,0.0,False,NaN,u,3,0,0,0,4145,84,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,449,55,527,61,561,68,0.00,0.00,NaN,NaN,1


In [1017]:
fpl_fixtures_df = pd.read_csv("fpl_historical_fixtures.csv")

In [1018]:
fpl_fixtures_df.head()

,gameweek,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,1,2444470,1,True,True,1,2024-08-16T19:00:00Z,90,False,True,9,0,14,1,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,3,115827
1,1,2444473,1,True,True,4,2024-08-17T11:30:00Z,90,False,True,12,2,10,0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,115830
2,1,2444471,1,True,True,2,2024-08-17T14:00:00Z,90,False,True,20,0,1,2,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828
3,1,2444472,1,True,True,3,2024-08-17T14:00:00Z,90,False,True,5,3,8,0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,3,115829
4,1,2444474,1,True,True,5,2024-08-17T14:00:00Z,90,False,True,17,0,15,1,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,115831


 Add team names

In [1020]:
# # Map team names back to their IDs
# team_mapping = {
#         1: 'Arsenal', 2: 'Aston Villa', 3: 'Bournemouth', 4: 'Brentford',
#         5: 'Brighton', 6: 'Chelsea', 7: 'Crystal Palace', 8: 'Everton',
#         9: 'Fulham', 10: 'Ipswich Town', 11: 'Leicester City', 12: 'Liverpool',
#         13: 'Manchester City', 14: 'Manchester Utd', 15: 'Newcastle Utd', 16: 'Nottingham Forest',
#         17: 'Southampton', 18: 'Tottenham', 19: 'West Ham', 20: 'Wolves'
#     }
# # Add Team 1 and Team 2 columns by mapping team_h and team_a using the team_mapping dictionary
# fpl_fixtures_df['Team 1'] = fpl_fixtures_df['team_h'].map(team_mapping)
# fpl_fixtures_df['Team 2'] = fpl_fixtures_df['team_a'].map(team_mapping)


Get data separately

In [1022]:
# Define the range of gameweeks
gameweeks = range(1, 18)  # Adjust as per the current number of gameweeks

# Initialize dictionaries to store gameweek-specific DataFrames
fixtures_dfs = {}
players_dfs = {}

# Loop through each gameweek and create DataFrames
for gw in gameweeks:
    # Filter fixtures for the current gameweek
    fixtures_dfs[gw] = fpl_fixtures_df[fpl_fixtures_df['gameweek'] == gw]
    
    # Filter players for the current gameweek
    players_dfs[gw] = fpl_players_df[fpl_players_df['gameweek'] == gw]

    # Optionally, save to CSV files if needed
    fixtures_dfs[gw].to_csv(f'fixtures_df{gw}.csv', index=False)
    players_dfs[gw].to_csv(f'fpl_players_df{gw}.csv', index=False)

# Example: Accessing a specific gameweek DataFrame
fixtures_df1 = fixtures_dfs[1]  # Fixtures for Gameweek 1
players_df1 = players_dfs[1]    # Players for Gameweek 1

# Print summary for confirmation
print(f"Created {len(fixtures_dfs)} fixtures DataFrames and {len(players_dfs)} players DataFrames.")


Created 17 fixtures DataFrames and 17 players DataFrames.


Process scores through function

In [1024]:
def process_gameweek_data(fixtures_df, players_df):
    def calculate_average_fdr():
        remaining_fixtures = fixtures_df[fixtures_df['finished'] == True]
        fdr_values = []
        remaining_fixtures = remaining_fixtures.sort_values(by='kickoff_time')
        for team in set(remaining_fixtures['team_h']).union(set(remaining_fixtures['team_a'])):
            team_fixtures = remaining_fixtures[
                (remaining_fixtures['team_h'] == team) | (remaining_fixtures['team_a'] == team)
            ].head(1) 
            for idx, fixture in team_fixtures.iterrows():
                if fixture['team_h'] == team:
                    fdr_values.append({'team': team, 'fdr': fixture['team_h_difficulty']})
                elif fixture['team_a'] == team:
                    fdr_values.append({'team': team, 'fdr': fixture['team_a_difficulty']})
        fdr_df = pd.DataFrame(fdr_values)
        average_fdr = fdr_df.groupby('team')['fdr'].mean().reset_index()
        average_fdr.columns = ['team_id', 'average_fdr']
        return average_fdr
    
    def add_average_fdr(players_df):
            # Get average FDR for each team
            average_fdr = calculate_average_fdr()
        
            # Map team names back to their IDs
            team_mapping = {
                1: 'Arsenal', 2: 'Aston Villa', 3: 'Bournemouth', 4: 'Brentford',
                5: 'Brighton', 6: 'Chelsea', 7: 'Crystal Palace', 8: 'Everton',
                9: 'Fulham', 10: 'Ipswich', 11: 'Leicester', 12: 'Liverpool',
                13: 'Man City', 14: 'Man Utd', 15: 'Newcastle', 16: "Nott'm Forest",
                17: 'Southampton', 18: 'Spurs', 19: 'West Ham', 20: 'Wolves',
            }
            
            # Map team names
            average_fdr['team_name'] = average_fdr['team_id'].map(team_mapping)
            # Original team mapping
            
            # Reverse the mapping
            reversed_team_mapping = {name: id_ for id_, name in team_mapping.items()}
            
            # Map the 'team' column
            players_df['team_id'] = players_df['team'].map(reversed_team_mapping)
    
            # Invert the average FDR
            max_fdr = average_fdr['average_fdr'].max()
            average_fdr['adjusted_fdr'] = 7 - average_fdr['average_fdr']
            # average_fdr.drop(columns = ['web_name'])
    
            players_df = players_df.merge(
            average_fdr[['team_id', 'adjusted_fdr', 'average_fdr']],
            how='left',
            on='team_id',
            suffixes=('', '_avg')
            )
                # Check for and drop any duplicates
            players_df = players_df.loc[:, ~players_df.columns.duplicated()]
    
            return players_df
        
    irrelevant_columns_players = [
        'squad_number',
        'region',
        'corners_and_indirect_freekicks_order',
        'chance_of_playing_this_round',
        'news_added',
        'chance_of_playing_next_round',
        'code', 'first_name', 'second_name', 'photo', 'news','direct_freekicks_text',
        'penalties_text','transfers_in','transfers_out'
    ]
    players_df_cleaned = players_df.drop(columns=irrelevant_columns_players, errors='ignore')
    players_df_cleaned['direct_freekicks_order'] = players_df_cleaned['direct_freekicks_order'].fillna(0)
    players_df_cleaned['penalties_order'] = players_df_cleaned['penalties_order'].fillna(0)
    players_df = players_df[~players_df['status'].isin(['i', 'u'])]
    players_df_cleaned = players_df_cleaned[~players_df_cleaned['status'].isin(['i', 'u'])]
    missing_scores_df = fixtures_df[(fixtures_df['team_h_score'].isnull()) | (fixtures_df['team_a_score'].isnull())]
    players_df_cleaned['value'] = players_df_cleaned['total_points'] / players_df_cleaned['now_cost']
    if 'status' in players_df.columns:
        le = LabelEncoder()
        players_df_cleaned['status'] = le.fit_transform(players_df_cleaned['status'])
    object_bool_columns = players_df_cleaned.select_dtypes(include=['object', 'bool'])
    bool_columns = players_df_cleaned.select_dtypes(include=['bool']).columns
    players_df_cleaned[bool_columns] = players_df_cleaned[bool_columns].astype(int)
    object_columns = players_df_cleaned.select_dtypes(include=['object']).columns
    players_df_cleaned[object_columns] = players_df_cleaned[object_columns].apply(pd.to_numeric, errors='coerce')
    data = players_df_cleaned.select_dtypes(include=['float64', 'int64']) 
    variances = data.var()
    sorted_variances = variances.sort_values(ascending=False)
    selector = VarianceThreshold(threshold=0)
    selector.fit(data)
    high_variance_columns = data.columns[selector.get_support()]
    high_variance_columns = [col for col in high_variance_columns if col != 'id']
    data = data[high_variance_columns]
    high_variance_sorted = sorted_variances[high_variance_columns].sort_values(ascending=False)
    data = data[high_variance_sorted.index]
    missing_cols = data.columns[data.isnull().any()]
    # Filter 'data' to align with 'players_df'
    data = data.loc[players_df.index]
    
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    data_scaled_df = pd.DataFrame(data_scaled, columns=data.columns)
    # Align data_scaled_df with players_df
    data_scaled_df = data_scaled_df.set_index(players_df.index)
        
    pca = PCA(n_components=2)  
    data_pca = pca.fit_transform(data_scaled_df)
    explained_variance = pca.explained_variance_ratio_
    explained_variance_pc1 = round(pca.explained_variance_ratio_[0],2)  
    explained_variance_pc2 = round(pca.explained_variance_ratio_[1],2)
    # Create aligned data_pca with the same index as players_df
    data_pca = pd.DataFrame(data=data_pca, index=players_df.index, columns=['PC1', 'PC2'])
    
    # Create pca_df with aligned indices
    pca_df = pd.DataFrame(data_pca)
    
    pca_df['Total Points'] = players_df['total_points']
    pca_df['web_name'] = players_df['web_name']
    pca_df['team'] = players_df['team']
    # Handle missing values in pca_df
    pca_df['Total Points'] = pca_df['Total Points'].fillna(0)
    pca_df['web_name'] = pca_df['web_name'].fillna(players_df['web_name'])
    pca_df['team'] = pca_df['team'].fillna(players_df['team'])
    # Continue with PCA regression and scoring
    pca_df_cleaned = pca_df.copy()
    pca_df_cleaned['PC1'] = pca_df_cleaned['PC1'].fillna(0)
    pca_df_cleaned['Total Points'] = pca_df_cleaned['Total Points'].fillna(0)
    
    X = pca_df_cleaned[['PC1']]
    y = pca_df_cleaned['Total Points']
    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)
    poly_model = LinearRegression()
    poly_model.fit(X_poly, y)
    poly_predictions = poly_model.predict(X_poly)
    poly_r2 = r2_score(y, poly_predictions)
    linear_model = LinearRegression()
    linear_model.fit(X, y)
    linear_predictions = linear_model.predict(X)
    linear_r2 = r2_score(y, linear_predictions)
    threshold = 0.01  
    non_linearity = 1 if (poly_r2 - linear_r2 > threshold) else 0
    non_linearity_pc1 = 1 if poly_r2 > linear_r2 else 0
    # Calculate Scores based on non-linearity
    if non_linearity_pc1 == 1:
        print("here",non_linearity_pc1)
        # PC1 is non-linear; use polynomial features
        poly = PolynomialFeatures(degree=2)  # Adjust degree as needed
        poly_features = poly.fit_transform(pca_df[['PC1']])  # This will include the constant term
    
        # Add polynomial features to the DataFrame
        pca_df[['PC1', 'PC1^2']] = poly_features[:, 1:]  # Ignore the first column (constant)
    
        # Calculate Score using polynomial feature of PC1 and original PC2
        players_df['Score'] = (explained_variance_pc1 * pca_df['PC1^2']) + (explained_variance_pc2 * pca_df['PC2'])
    else:
        # PC1 is linear; use original PC1
        players_df['Score'] = (explained_variance_pc1 * pca_df['PC1']) + (explained_variance_pc2 * pca_df['PC2'])

    inertia = []  # To store the inertia values
    k_values = range(1, 11)  # Adjust the range based on your needs
    for k in k_values:
        kmeans = KMeans(n_clusters=k, n_init = 10, random_state=42)  # Using random_state for reproducibility
        kmeans.fit(data_scaled_df)  # Fit the model
        inertia.append(kmeans.inertia_)  # Store the inertia value
    optimal_k = 2
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)  # Using random_state for reproducibility
    if non_linearity_pc1:
        kmeans.fit(pca_df[['PC1','PC1^2', 'PC2']])  # Fit the model
    else:
        kmeans.fit(pca_df[['PC1', 'PC2']])  # Fit the model
    
    # Add cluster labels to your original DataFrame
    pca_df['Cluster'] = kmeans.labels_
    
    pca_df['web_name'] = players_df['web_name']  # Add the player names to the PCA DataFrame
    print("hello",pca_df[pca_df['web_name'].isin(players_df[players_df['team'] == "Wolves"]['web_name'])])


    players_with_clusters = players_df.merge(pca_df[['web_name', 'Cluster', 'PC1']], on='web_name', how='left')
    print( players_with_clusters.head())
    # Calculate the average 'PC1' value for each cluster
    cluster_avg = players_with_clusters.groupby('Cluster')['PC1'].mean()
    
    # Identify the worst cluster (lowest average 'PC1')
    worst_cluster = cluster_avg.idxmin()
    
    # Filter out players from the worst cluster
    players_df_filtered = players_with_clusters[players_with_clusters['Cluster'] != worst_cluster]


    players_df_filtered = add_average_fdr(players_df_filtered)
    weight = 2
    players_df_filtered['adjusted_score'] = players_df_filtered['Score'] + (weight * players_df_filtered['adjusted_fdr'])
    players_df_filtered_sorted = players_df_filtered.sort_values(by='Score', ascending=False)
    position_mapping = {1: 'goalkeeper', 2: 'defender', 3: 'midfielder', 4: 'forward'}
    players_df_filtered_sorted['position'] = players_df_filtered_sorted['element_type'].map(position_mapping)
    players_df_filtered_sorted = add_average_fdr(players_df_filtered_sorted)

  

    return players_df_filtered_sorted


In [1025]:
# Initialize results_df as an empty dictionary or DataFrame
results_df = {}  # Use a dictionary to store results for each gameweek

# Loop through all gameweeks
for gw in range(1, 18):  # Adjust range as needed for all gameweeks
    fixtures_dfs[gw].to_csv(f'fixtures_df{gw}.csv', index=False)
    players_dfs[gw].to_csv(f'fpl_players_df{gw}.csv', index=False)
    
    fixtures_df = pd.read_csv(f'fixtures_df{gw}.csv')  # Get fixtures DataFrame for the gameweek
    players_df = pd.read_csv(f'fpl_players_df{gw}.csv')   # Get players DataFrame for the gameweek    
    # Call the function for the gameweek
    results_df[gw] = process_gameweek_data(fixtures_df, players_df)
    

# After the loop, you can combine the results into a single DataFrame if needed
final_results_df = pd.concat(results_df.values(), ignore_index=True)
# Replace NaN values in the 'adjusted_score' column with 0
final_results_df['adjusted_score'] = final_results_df['adjusted_score'].fillna(0)


/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:120: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)
/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['Score'] = (explained_variance_pc1 * pca_df['PC1^2']) + (explained_variance_pc2 * pca_df['PC2'])


here 1
hello           PC1       PC2  Total Points       web_name    team     PC1^2  \
203 -0.322633 -0.644733            12      Johnstone  Wolves  0.104092   
532  0.929787 -0.502724            40      Aït-Nouri  Wolves  0.864504   
534  0.076475  0.070000            33     Bellegarde  Wolves  0.005848   
535 -1.028011 -0.321118             0        Bentley  Wolves  1.056807   
538 -1.089204  0.236775             0        Chiwome  Wolves  1.186366   
539 -1.357736  0.220875             0         Cundle  Wolves  1.843447   
540  2.337967  0.877732            98          Cunha  Wolves  5.466091   
541 -0.423489 -0.401020            15         Dawson  Wolves  0.179343   
542 -0.045986 -0.110911            20        Doherty  Wolves  0.002115   
543 -0.721742  0.232876            18          Doyle  Wolves  0.520912   
547  0.137278  0.312294            39         Guedes  Wolves  0.018845   
549 -0.764869  0.155136            12       Hee Chan  Wolves  0.585025   
552  0.902549 -0.286038  

/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['team_id'] = players_df['team'].map(reversed_team_mapping)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:120: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)


here 1


/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['Score'] = (explained_variance_pc1 * pca_df['PC1^2']) + (explained_variance_pc2 * pca_df['PC2'])


...

/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['team_id'] = players_df['team'].map(reversed_team_mapping)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:120: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)
/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


here 1
hello           PC1       PC2  Total Points       web_name    team     PC1^2  \
203 -0.133711 -0.733297            12      Johnstone  Wolves  0.017879   
532  1.085074 -0.344976            40      Aït-Nouri  Wolves  1.177386   
534  0.238953  0.170005            33     Bellegarde  Wolves  0.057099   
535 -0.848137 -0.570161             0        Bentley  Wolves  0.719337   
538 -0.925987  0.050595             0        Chiwome  Wolves  0.857453   
539 -1.212459  0.194771             0         Cundle  Wolves  1.470058   
540  2.439023  0.923197            98          Cunha  Wolves  5.948832   
541 -0.264753 -0.230516            15         Dawson  Wolves  0.070094   
542  0.130101 -0.171073            20        Doherty  Wolves  0.016926   
543 -0.560520  0.262240            18          Doyle  Wolves  0.314182   
547  0.305596  0.224097            39         Guedes  Wolves  0.093389   
549 -0.596750  0.115675            12       Hee Chan  Wolves  0.356110   
552  1.066653 -0.093817  

/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['team_id'] = players_df['team'].map(reversed_team_mapping)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_variance_threshold.py:120: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)
/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


here 1
hello           PC1       PC2  Total Points       web_name    team     PC1^2  \
203 -0.133711 -0.733297            12      Johnstone  Wolves  0.017879   
532  1.085074 -0.344976            40      Aït-Nouri  Wolves  1.177386   
534  0.238953  0.170005            33     Bellegarde  Wolves  0.057099   
535 -0.848137 -0.570161             0        Bentley  Wolves  0.719337   
538 -0.925987  0.050595             0        Chiwome  Wolves  0.857453   
539 -1.212459  0.194771             0         Cundle  Wolves  1.470058   
540  2.439023  0.923197            98          Cunha  Wolves  5.948832   
541 -0.264753 -0.230516            15         Dawson  Wolves  0.070094   
542  0.130101 -0.171073            20        Doherty  Wolves  0.016926   
543 -0.560520  0.262240            18          Doyle  Wolves  0.314182   
547  0.305596  0.224097            39         Guedes  Wolves  0.093389   
549 -0.596750  0.115675            12       Hee Chan  Wolves  0.356110   
552  1.066653 -0.093817  

/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_34355/2271902540.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['team_id'] = players_df['team'].map(reversed_team_mapping)


In [1026]:
final_results_df.to_csv("static_fpl_scores.csv")

In [1027]:
final_results_df['adjusted_score'].describe()

count    1753.000000
mean        8.947110
std         2.090554
min         0.000000
25%         7.283247
50%         8.988138
75%        10.681624
max        15.657131
Name: adjusted_score, dtype: float64

In [1028]:
final_results_df.columns

Index(['gameweek', 'id', 'web_name', 'team', 'position', 'can_transact',
       'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo',
       'points_per_game', 'removed', 'second_name', 'selected_by_percent',
       'special', 'squad_number', 'status', 'team_code', 'total_points',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'region',
       'team_join_date', 'minutes', 'goals_scored', 'assists', 'clean_sheets',
       'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed',
       'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence',
       'creativity', 'threat', 'ict_index', 'starts', 'expected_

In [1029]:
wolves_web_names = final_results_df[final_results_df['team'] == 'Wolves']['element_type'].unique()
print(wolves_web_names)

[4 2 3]


In [1030]:
len(final_results_df)

1753

In [1031]:
# Group by 'gameweek' and 'team', calculate the average 'adjusted_score'
grouped_results_df = final_results_df.groupby(['gameweek', 'team'], as_index=False).agg(
    avg_adjusted_score=('adjusted_score', 'mean')
)
grouped_results_df.iloc[0:50]

,gameweek,team,avg_adjusted_score
0,1,Arsenal,10.830699
1,1,Aston Villa,10.834937
2,1,Bournemouth,9.219525
3,1,Brentford,9.294263
4,1,Brighton,8.830482
5,1,Chelsea,7.629526
6,1,Crystal Palace,6.666739
7,1,Everton,8.953171
8,1,Fulham,8.974790
9,1,Ipswich,4.475801


In [1032]:
# Filtering players_df to show all rows where the team is 'Wolves'
wolves_players_df = grouped_results_df[grouped_results_df['team'] == "Spurs"]
wolves_players_df.head()

,gameweek,team,avg_adjusted_score
17,1,Spurs,11.323699
37,2,Spurs,11.319336
57,3,Spurs,7.368599
77,4,Spurs,7.420405
97,5,Spurs,11.428867


In [1033]:
players_df['team'].unique()

array(['Arsenal', 'Fulham', 'Crystal Palace', 'Southampton',
       "Nott'm Forest", 'Aston Villa', 'Bournemouth', 'Spurs',
       'Brentford', 'Brighton', 'Leicester', 'Chelsea', 'Everton',
       'Wolves', 'Ipswich', 'Liverpool', 'Man City', 'Man Utd',
       'West Ham', 'Newcastle'], dtype=object)